# 项目：评估和清理英国电商公司销售数据

## 分析目标

此数据分析的目的是，根据市场销售数据，挖掘畅销产品，以便制定更有效的市场策略来提升营收。

本实战项目的目的在于练习评估数据干净和整洁度，并且基于评估结果，对数据进行清洗，从而得到可供下一步分析的数据。

## 简介

原始数据集记录了一家英国在线零售公司在2010年12月1日至2011年12月9日期间的所有交易情况，涵盖了该公司在全球不同国家和地区的业务数据。该公司主要销售覆盖各个场景的礼品，包括但不限于生日礼品、结婚纪念品、圣诞礼品等等。该公司的客户群体主要包括批发商和个人消费者，其中批发商占据了相当大的比例。

数据每列的含义如下：
- `InvoiceNo`: 发票号码。6位数，作为交易的唯一标识符。如果这个代码以字母“c”开头，表示这笔交易被取消。
- `StockCode`: 产品代码。5位数，作为产品的唯一标识符。
- `Description`: 产品名称。
- `Quantity`: 产品在交易中的数量。
- `InvoiceDate`: 发票日期和时间。交易发生的日期和时间。
- `UnitPrice`: 单价。价格单位为英镑（£）。
- `CustomerID`: 客户编号。5位数，作为客户的唯一标识符。
- `Country`: 国家名称。客户所居住的国家的名称。

## 读取数据

In [10]:
import pandas as pd

In [11]:

original_data = pd.read_csv("e_commerce.csv")
original_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


## 评估数据

In [12]:
在这一部分，我将对在上一部分建立的original_data这个DataFrame所包含的数据进行评估。

评估主要从两个方面进行：结构和内容，即整齐度和干净度。数据的结构性问题指不符合“每列是一个变量，每行是一个观察值，
每个单元格是一个值”这三个标准，数据的内容性问题包括存在丢失数据、重复数据、无效数据等

SyntaxError: invalid character '，' (U+FF0C) (362356746.py, line 1)

评估数据整齐度

In [ ]:
original_data.sample(10)

In [ ]:
从抽样的10行数据数据来看，数据符合“每列是一个变量，每行是一个观察值，每个单元格是一个值”，具体来看每行是关于某商品的一次交易，
每列是交易相关的各个变量，因此不存在结构性问题。

In [ ]:
评估数据干净度

In [ ]:
original_data.info()

In [ ]:
(1)从输出结果来看，数据共有541909条观察值，而Description、CustomerID变量存在缺失值。

(2)此外，InvoiceDate的数据类型应为日期，CustomerID的数据类型应为字符串，应当进行数据格式转换。

In [ ]:
评估缺失数据

In [ ]:
original_data[original_data["Description"].isnull()]

In [ ]:
有1454条交易数据缺失Description变量值。

从输出结果来看，这些缺失Description的交易数据，UnitPrice都为0。为了验证猜想，我们增加筛选条件，看是否存在Description变量缺失且UnitPrice不为0的数据

In [ ]:
original_data[(original_data["Description"].isnull())&(original_data["UnitPrice"]!=0)]

In [ ]:
筛选出来结果数量为0条，说明缺失`Description`值的数据，同时也不具备有效的`UnitPrice`值。

`Description`表示产品名称，`UnitPrice`表示产品单价，都是进行后续商品交易分析的重要变量。如果它们同时缺失/无效，我们认为数据无法提供有效含义，因此这些后续可以被删除。

In [ ]:
CustomerID变量同样存在缺失值，因此也根据条件提取出缺失观察值。

In [ ]:
original_data[original_data["CustomerID"].isnull()]

In [ ]:
CustomerID表示客户编号，不是分析畅销商品的必要变量。并且从输出结果来看，有些CustomerID缺失的交易数据仍然有效，因此保留此变量为空的观察值。

In [ ]:
评估重复数据

In [ ]:
根据数据变量的含义来看，虽然InvoiceNo、StockCode和CustomerID都是唯一标识符，
但一次交易可能包含多件商品，因此InvoiceNo可以存在重复，不同交易可以包含同件商品，因此StockCode可以存在重复，
顾客可以进行多次交易或下单多个商品，因此CustomerID也可以存在重复。

那么针对此数据集，我们无需评估重复数据

In [ ]:
评估不一致数据


In [ ]:
不一致数据可能存在于Country变量中，我们要查看是否存在多个不同值指代同一国家的情况。

In [ ]:
original_data["Country"].value_counts()

In [ ]:
从Country变量值来看，"USA"、"United States"均在表示美国，"United Kingdom"、"UK"、"U.K."均在表示英国，因此应该在清洗步骤对这些值进行统一，只保留一个指代值。

In [ ]:
评估无效和错误数据

In [ ]:
original_data.describe()

In [ ]:
从输出结果来看，Quantity和UnitPrice存在负数，会对后续数值分析产生影响。

因此，我们先筛选出Quantity数值为负数的观察值，进一步评估其含义。

In [ ]:
original_data[(original_data["Quantity"]<0)&(original_data["InvoiceNo"].str[0]!="C")]


In [ ]:
以上猜想被证实错误，因为还存在Quantity变量为负数且InvoiceNo不以C开头的观察值。

但根据以上输出结果，这些筛选出的观察值的UnitPrice观察值均为0，因此增加UnitPrice的条件进行验证。

In [ ]:
original_data[(original_data["Quantity"] < 0)&(original_data["InvoiceNo"].str[0] != "C")&(original_data["UnitPrice"] != 0)]

In [ ]:
根据输出结果，猜想得到验证，当Quantity变量为负数的时候，观察值满足以下条件之一：

InvoiceNo以C开头，表示订单被取消
UnitPrice为0，说明单价为0英镑
这些交易数据均不是有效成交数据，不贡献销售，不在后续分析范围内，因此我们将在数据清理步骤，将Quantity为负数的观察值删除。

In [ ]:
original_data[original_data["UnitPrice"]<0]

In [ ]:
从输出结果来看，UnitPrice为负数的观察值都是坏账调账，不属于实际商品交易数据，因此也在数据清理步骤中也将其删除

In [ ]:
清洗数据

In [ ]:
根据前面评估部分得到的结论，我们需要进行的数据清理包括：

1.把InvoiceDate变量的数据类型转换为为日期时间
2.把CustomerID变量的数据类型转换为字符串
3.把Description变量缺失的观察值删除
4.把Country变量值"USA"替换为"United States"
5.把Country变量值"UK"、"U.K."替换为"United Kingdom"
6.把Quantity变量值为负数的观察值删除
7.把UnitPrice变量值为负数的观察值删除

In [ ]:
为了区分开经过清理的数据和原始的数据，我们创建新的变量cleaned_data，让它为original_data复制出的副本。我们之后的清理步骤都将被运用在cleaned_data上。

In [33]:
cleaned_data=original_data.copy()

In [ ]:
(1)

In [ ]:
cleaned_data["InvoiceDate"]=pd.to_datetime(cleaned_data["InvoiceDate"])

In [ ]:
cleaned_data["InvoiceDate"]

In [ ]:
（2）


In [ ]:
cleaned_data["CustomerID"]=cleaned_data["CustomerID"].astype(str)
cleaned_data["CustomerID"]

In [ ]:
把CustomerID变量值结尾的".0"删除

In [ ]:
cleaned_data["CustomerID"].str.slice(0,-2)


In [ ]:
(3)

In [ ]:
把Description变量缺失的观察值删除，把Description变量缺失的观察值删除，
并查看删除后该列空缺值个数和并查看删除后该列空缺值个数和把Description变量缺失的观察值删除，并查看删除后该列空缺值个数和

In [18]:
cleaned_data=cleaned_data.dropna(subset=["Description"])

In [19]:
cleaned_data["Description"].isnull().sum()

np.int64(0)

In [ ]:
(4)把Country变量值"USA"替换为"United States"，并检查替换后"USA"变量值个数：

In [34]:
cleaned_data["Country"]=cleaned_data["Country"].replace({"USA":"United States"})



In [35]:


len(cleaned_data[cleaned_data["Country"] == "USA"])

0

In [ ]:
把Country变量值"UK"、"U.K."替换为"United Kingdom"，并检查替换后"UK"和"U.K."变量值个数：

In [36]:
cleaned_data["Country"]=cleaned_data["Country"].replace({"UK":"United Kindom","U.K.":"United Kindom"})

In [38]:
len(cleaned_data[cleaned_data["Country"]=="UK"])

0

In [39]:
len(cleaned_data[cleaned_data["Country"]=="U.K."])

0

In [ ]:
(6)

In [40]:
cleaned_data=cleaned_data[cleaned_data["Quantity"]>=0]

In [41]:
cleaned_data=cleaned_data[cleaned_data["UnitPrice"]>=0]


In [46]:
len(cleaned_data[cleaned_data["Quantity"]<0])

0

## 保存清理后的数据

In [49]:

cleaned_data.to_csv("e_commerce_cleaned.csv",index=False)

In [51]:
cleaned_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [50]:
pd.read_csv("e_commerce_cleaned.csv").head()

C:\Users\86137\AppData\Local\Temp\ipykernel_23492\2686214642.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv("e_commerce_cleaned.csv").head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
